In [1]:
import os
ROOT = os.path.join('..', '..')
import sys
sys.path.append(ROOT)
# 
from tqdm import tqdm
import numpy as np
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()

from scripts.data import (
    ml_ratings_df, ml_movies_df, ml_users_df, ml_train_test_split
)

from src.constants import T_SPLIT_POINT

ml_ratings_df = ml_ratings_df.collect().to_pandas()
ml_users_df = ml_users_df.collect().to_pandas()
ml_movies_df = ml_movies_df.collect().to_pandas()

# Train - Test Split

In [2]:
MAX_K = 30
ml_ratings_train_df, ml_ratings_test_df = ml_train_test_split(
    ml_ratings_df, MAX_K, T_SPLIT_POINT
)
print('Train shape:', ml_ratings_train_df.shape)
print('Test shape:', ml_ratings_test_df.shape)

Train shape: (85086, 4)
Test shape: (534709, 4)


# Item-Item Colaborative Filtering